<a href="https://colab.research.google.com/github/nirmal-ank/PINN-Learning/blob/main/Fuselage_Panel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

For permanent install

In [ ]:
# !pip install virtualenv
# !virtualenv /content/drive/MyDrive/virtual_env
#  !source /content/drive/MyDrive/virtual_env/bin/activate; pip install nvidia-modulus nvidia-modulus-sym


Add the installation path of the modulus to the system


In [ ]:
import sys
# add the path of the virtual environmentsite-packages to colab system path
sys.path.append("/content/drive/MyDrive/virtual_env/lib/python3.10/site-packages")

For temporary install

In [ ]:
# !pip install nvidia-modulus nvidia-modulus-sym


# Import Required Libraries

In [ ]:
import os
import warnings

import numpy as np

from sympy import Symbol, Eq

import modulus.sym
# from modulus.sym.hydra import to_absolute_path, instantiate_arch, ModulusConfig
from modulus.sym.utils.io import csv_to_dict
from modulus.sym.solver import Solver
from modulus.sym.domain import Domain
from modulus.sym.geometry.primitives_2d import Rectangle, Circle
from modulus.sym.domain.constraint import (
    PointwiseBoundaryConstraint,
    PointwiseInteriorConstraint,
)
from modulus.sym.domain.validator import PointwiseValidator
from modulus.sym.domain.inferencer import PointwiseInferencer
from modulus.sym.key import Key
from modulus.sym.node import Node


2024-08-16 06:31:22.428799: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-16 06:31:22.637171: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-16 06:31:22.716603: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-16 06:31:22.739103: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-16 06:31:22.894087: I tensorflow/core/platform/cpu_feature_guar

Linear Elasticity Equations

In [ ]:
from modulus.sym.eq.pdes.linear_elasticity import LinearElasticityPlaneStress

Panel Properties

In [ ]:
E = 73.0 * 10**9  # Pa
nu = 0.33
lambda_ = nu * E / ((1 + nu) * (1 - 2 * nu))  # Pa
mu_real = E / (2 * (1 + nu))  # Pa
lambda_ = lambda_ / mu_real  # Dimensionless
mu = 1.0  # Dimensionless

Creating pde equation node for linear elasticity

In [ ]:
le = LinearElasticityPlaneStress(lambda_=lambda_, mu=mu)

In [ ]:
le.pprint()

stress_disp_xx: -sigma_xx + 2.98507462686567*u__x + 0.985074626865672*v__y
stress_disp_yy: -sigma_yy + 0.985074626865672*u__x + 2.98507462686567*v__y
stress_disp_xy: -sigma_xy + 1.0*u__y + 1.0*v__x
equilibrium_x: -sigma_xx__x - sigma_xy__y
equilibrium_y: -sigma_xy__x - sigma_yy__y
traction_x: normal_x*sigma_xx + normal_y*sigma_xy
traction_y: normal_x*sigma_xy + normal_y*sigma_yy


In [ ]:
le.equations["stress_disp_xx"]


In [ ]:
le.equations["stress_disp_yy"]


In [ ]:
le.equations["stress_disp_xy"]


In [ ]:
le.equations["equilibrium_x"]


In [ ]:
le.equations["equilibrium_y"]


In [ ]:
le.equations["traction_x"]


In [ ]:
le.equations["traction_y"]

# Creating Neural Network Node

In [ ]:
from modulus.sym.models.fully_connected import FullyConnectedArch
from modulus.sym.models.activation import Activation
from modulus.sym.key import Key

elasticity_net = FullyConnectedArch(
      input_keys=[Key("x"), Key("y"), Key("sigma_hoop")],
      output_keys=[
        Key("u"),
        Key("v"),
        Key("sigma_xx"),
        Key("sigma_yy"),
        Key("sigma_xy"),
        ],
      layer_size=512,
      nr_layers=6,
      skip_connections=False,
      activation_fn=Activation.SILU,
      adaptive_activations=False,
      weight_norm=True,
)

In [ ]:
elasticity_net

FullyConnectedArch(
  (_impl): FullyConnectedArchCore(
    (layers): ModuleList(
      (0): FCLayer(
        (linear): WeightNormLinear(in_features=3, out_features=512, bias=True)
      )
      (1-5): 5 x FCLayer(
        (linear): WeightNormLinear(in_features=512, out_features=512, bias=True)
      )
    )
    (final_layer): FCLayer(
      (activation_fn): Identity()
      (linear): Linear(in_features=512, out_features=5, bias=True)
    )
  )
)

In [ ]:
nodes = le.make_nodes() + [elasticity_net.make_node(name="elasticity_network")]

In [ ]:
nodes

# Geometry

In [ ]:
x, y, sigma_hoop = Symbol("x"), Symbol("y"), Symbol("sigma_hoop")
panel_origin = (-0.5, -0.9)
panel_dim = (1, 1.8)  # Panel width is the characteristic length.
window_origin = (-0.125, -0.2)
window_dim = (0.25, 0.4)
panel_aux1_origin = (-0.075, -0.2)
panel_aux1_dim = (0.15, 0.4)
panel_aux2_origin = (-0.125, -0.15)
panel_aux2_dim = (0.25, 0.3)
hr_zone_origin = (-0.2, -0.4)
hr_zone_dim = (0.4, 0.8)
circle_nw_center = (-0.075, 0.15)
circle_ne_center = (0.075, 0.15)
circle_se_center = (0.075, -0.15)
circle_sw_center = (-0.075, -0.15)
circle_radius = 0.05
panel = Rectangle(
    panel_origin, (panel_origin[0] + panel_dim[0], panel_origin[1] + panel_dim[1])
)
window = Rectangle(
    window_origin,
    (window_origin[0] + window_dim[0], window_origin[1] + window_dim[1]),
)
panel_aux1 = Rectangle(
    panel_aux1_origin,
    (
        panel_aux1_origin[0] + panel_aux1_dim[0],
        panel_aux1_origin[1] + panel_aux1_dim[1],
    ),
)
panel_aux2 = Rectangle(
    panel_aux2_origin,
    (
        panel_aux2_origin[0] + panel_aux2_dim[0],
        panel_aux2_origin[1] + panel_aux2_dim[1],
    ),
)
hr_zone = Rectangle(
    hr_zone_origin,
    (hr_zone_origin[0] + hr_zone_dim[0], hr_zone_origin[1] + hr_zone_dim[1]),
)
circle_nw = Circle(circle_nw_center, circle_radius)
circle_ne = Circle(circle_ne_center, circle_radius)
circle_se = Circle(circle_se_center, circle_radius)
circle_sw = Circle(circle_sw_center, circle_radius)
corners = (
    window - panel_aux1 - panel_aux2 - circle_nw - circle_ne - circle_se - circle_sw
)
window = window - corners
geo = panel - window
hr_geo = geo & hr_zone

In [ ]:
hr_geo

In [ ]:
from modulus.sym.utils.io.vtk import var_to_polyvtk
samples = hr_geo.sample_boundary(100000)
var_to_polyvtk(samples, './hr_geo_bound')
samples = hr_geo.sample_interior(100000)
var_to_polyvtk(samples, './hr_geo_int')


Parameterization

In [ ]:
characteristic_length = panel_dim[0]
characteristic_disp = 0.001 * window_dim[0]
sigma_normalization = characteristic_length / (mu_real * characteristic_disp)
sigma_hoop_lower = 46 * 10**6 * sigma_normalization
sigma_hoop_upper = 56.5 * 10**6 * sigma_normalization
sigma_hoop_range = (sigma_hoop_lower, sigma_hoop_upper)
param_ranges = {sigma_hoop: sigma_hoop_range}
inference_param_ranges = {sigma_hoop: 46 * 10**6 * sigma_normalization}

Bounds

In [ ]:
    bounds_x = (panel_origin[0], panel_origin[0] + panel_dim[0])
    bounds_y = (panel_origin[1], panel_origin[1] + panel_dim[1])
    hr_bounds_x = (hr_zone_origin[0], hr_zone_origin[0] + hr_zone_dim[0])
    hr_bounds_y = (hr_zone_origin[1], hr_zone_origin[1] + hr_zone_dim[1])

# Constraints

## Domain

In [ ]:
    domain = Domain()

## Boundary Constraints

In [ ]:
# left wall
panel_left = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=geo,
    outvar={"traction_x": 0.0, "traction_y": 0.0},
    batch_size=250,
    criteria=Eq(x, panel_origin[0]),
    parameterization=param_ranges,
)
domain.add_constraint(panel_left, "panel_left")

# right wall
panel_right = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=geo,
    outvar={"traction_x": 0.0, "traction_y": 0.0},
    batch_size=250,
    criteria=Eq(x, panel_origin[0] + panel_dim[0]),
    parameterization=param_ranges,
)
domain.add_constraint(panel_right, "panel_right")

# bottom wall
panel_bottom = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=geo,
    outvar={"v": 0.0},
    batch_size=150,
    criteria=Eq(y, panel_origin[1]),
    parameterization=param_ranges,
)
domain.add_constraint(panel_bottom, "panel_bottom")

# corner point
panel_corner = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=geo,
    outvar={"u": 0.0},
    batch_size=5,
    criteria=Eq(x, panel_origin[0])
    & (y > panel_origin[1])
    & (y < panel_origin[1] + 1e-3),
    parameterization=param_ranges,
)
domain.add_constraint(panel_corner, "panel_corner")

# top wall
panel_top = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=geo,
    outvar={"traction_x": 0.0, "traction_y": sigma_hoop},
    batch_size=150,
    criteria=Eq(y, panel_origin[1] + panel_dim[1]),
    parameterization=param_ranges,
)
domain.add_constraint(panel_top, "panel_top")

# pannel window
panel_window = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=window,
    outvar={"traction_x": 0.0, "traction_y": 0.0},
    batch_size=3500,
    parameterization=param_ranges,
)
domain.add_constraint(panel_window, "panel_window")

## Interior Constraints

In [ ]:
# low-resolution interior
lr_interior = PointwiseInteriorConstraint(
    nodes=nodes,
    geometry=geo,
    outvar={
        "equilibrium_x": 0.0,
        "equilibrium_y": 0.0,
        "stress_disp_xx": 0.0,
        "stress_disp_yy": 0.0,
        "stress_disp_xy": 0.0,
    },
    batch_size=7000,
    bounds={x: bounds_x, y: bounds_y},
    lambda_weighting={
        "equilibrium_x": Symbol("sdf"),
        "equilibrium_y": Symbol("sdf"),
        "stress_disp_xx": Symbol("sdf"),
        "stress_disp_yy": Symbol("sdf"),
        "stress_disp_xy": Symbol("sdf"),
    },
    parameterization=param_ranges,
)
domain.add_constraint(lr_interior, "lr_interior")

# high-resolution interior
hr_interior = PointwiseInteriorConstraint(
    nodes=nodes,
    geometry=hr_geo,
    outvar={
        "equilibrium_x": 0.0,
        "equilibrium_y": 0.0,
        "stress_disp_xx": 0.0,
        "stress_disp_yy": 0.0,
        "stress_disp_xy": 0.0,
    },
    batch_size=4000,
    bounds={x: hr_bounds_x, y: hr_bounds_y},
    lambda_weighting={
        "equilibrium_x": Symbol("sdf"),
        "equilibrium_y": Symbol("sdf"),
        "stress_disp_xx": Symbol("sdf"),
        "stress_disp_yy": Symbol("sdf"),
        "stress_disp_xy": Symbol("sdf"),
    },
    parameterization=param_ranges,
)
domain.add_constraint(hr_interior, "hr_interior")

Add inference data

In [ ]:
invar_numpy = geo.sample_interior(
        100000,
        bounds={x: bounds_x, y: bounds_y},
        parameterization=inference_param_ranges,
    )
point_cloud_inference = PointwiseInferencer(
    nodes=nodes,
    invar=invar_numpy,
    output_names=["u", "v", "sigma_xx", "sigma_yy", "sigma_xy"],
    batch_size=4096,
)
domain.add_inferencer(point_cloud_inference, "inf_data")

# Solver Configuration

In [ ]:
import modulus.sym
from modulus.sym.hydra import to_yaml
from modulus.sym.hydra.utils import compose
from modulus.sym.hydra.config import ModulusConfig

cfg = compose(config_path=".", config_name="config")
cfg.network_dir = 'outputs' # Set the network directory for checkpoints
print(to_yaml(cfg))

/home/ank/miniconda3/envs/PINN/lib/python3.9/site-packages/modulus/sym/hydra/utils.py:150: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  hydra.initialize(


training:
  max_steps: 5000
  grad_agg_freq: 1
  rec_results_freq: 2000
  rec_validation_freq: ${training.rec_results_freq}
  rec_inference_freq: ${training.rec_results_freq}
  rec_monitor_freq: ${training.rec_results_freq}
  rec_constraint_freq: 10000
  save_network_freq: 1000
  print_stats_freq: 100
  summary_freq: 1000
  amp: false
  amp_dtype: float16
  ntk:
    use_ntk: false
    save_name: null
    run_freq: 1000
graph:
  func_arch: false
  func_arch_allow_partial_hessian: true
stop_criterion:
  metric: null
  min_delta: null
  patience: 50000
  mode: min
  freq: 1000
  strict: false
profiler:
  profile: false
  start_step: 0
  end_step: 100
  name: nvtx
network_dir: outputs
initialization_network_dir: ''
save_filetypes: vtk
summary_histograms: false
jit: true
jit_use_nvfuser: true
jit_arch_mode: only_activation
jit_autograd_nodes: false
cuda_graphs: true
cuda_graph_warmup: 20
find_unused_parameters: false
broadcast_buffers: false
device: ''
debug: false
run_mode: train
arch:
  f

# Create Solver

In [ ]:
from modulus.sym.solver import Solver

slv = Solver(cfg,domain)


# Run Solver

First configure the logging to see the steps!

In [ ]:
import logging

# Create a function to configure logging
def configure_logging():
    # Get the root logger
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)

    # Remove all existing handlers
    while logger.handlers:
        logger.handlers.pop()

    # Create a stream handler
    stream_handler = logging.StreamHandler()
    stream_handler.setLevel(logging.DEBUG)

    # Create a formatter
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    stream_handler.setFormatter(formatter)

    # Add the handler to the logger
    logger.addHandler(stream_handler)

# Configure logging
configure_logging()

# Test logging
logging.debug("Logging is configured correctly.")

2024-08-15 08:32:26,680 - root - DEBUG - Logging is configured correctly.


In [ ]:
slv.solve()

2024-08-15 08:32:28,725 - modulus.sym.trainer - WARNING - Installed PyTorch version 2.3.1+cu121 is not TorchScript supported in Modulus. Version 2.1.0a0+4136153 is officially supported.
2024-08-15 08:32:28,737 - modulus.sym.trainer - INFO - attempting to restore from: /content/outputs
2024-08-15 08:32:28,738 - modulus.sym.trainer - WARNING - optimizer checkpoint not found
2024-08-15 08:32:28,740 - modulus.sym.trainer - WARNING - model elasticity_network.0.pth not found
2024-08-15 08:32:29,299 - modulus.sym.trainer - DEBUG - [step:          0] saved constraint results to outputs
2024-08-15 08:32:29,300 - modulus.sym.trainer - INFO - [step:          0] record constraint batch time:  2.231e-01s
2024-08-15 08:32:29,719 - modulus.sym.trainer - DEBUG - [step:          0] saved inferencer results to outputs
2024-08-15 08:32:29,720 - modulus.sym.trainer - INFO - [step:          0] record inferencers time:  4.145e-01s
2024-08-15 08:32:29,753 - modulus.sym.trainer - INFO - [step:          0] sav

# Post Processing

## Load Model From File

In [ ]:
import torch
torch.load('./outputs/optim_checkpoint.0.pth').keys()

dict_keys(['step', 'optimizer_state_dict', 'aggregator_state_dict', 'scheduler_state_dict', 'scaler_state_dict'])

In [ ]:
torch.load('./outputs/elasticity_network.0.pth').keys()

odict_keys(['_impl.layers.0.linear.weight', '_impl.layers.0.linear.weight_g', '_impl.layers.0.linear.bias', '_impl.layers.1.linear.weight', '_impl.layers.1.linear.weight_g', '_impl.layers.1.linear.bias', '_impl.layers.2.linear.weight', '_impl.layers.2.linear.weight_g', '_impl.layers.2.linear.bias', '_impl.layers.3.linear.weight', '_impl.layers.3.linear.weight_g', '_impl.layers.3.linear.bias', '_impl.layers.4.linear.weight', '_impl.layers.4.linear.weight_g', '_impl.layers.4.linear.bias', '_impl.layers.5.linear.weight', '_impl.layers.5.linear.weight_g', '_impl.layers.5.linear.bias', '_impl.final_layer.linear.weight', '_impl.final_layer.linear.bias'])

In [ ]:
# checkpoint = torch.load('./outputs/optim_checkpoint.0.pth')
checkpoint = torch.load('outputs/elasticity_network.0.pth')
try:
    checkpoint.eval()
except AttributeError as error:
    print(error)



elasticity_net.load_state_dict(checkpoint)

'collections.OrderedDict' object has no attribute 'eval'


NameError: name 'elasticity_net' is not defined

## How to evaluate a PyTorch model

In [ ]:
elasticity_net

FullyConnectedArch(
  (_impl): FullyConnectedArchCore(
    (layers): ModuleList(
      (0): FCLayer(
        (linear): WeightNormLinear(in_features=2, out_features=512, bias=True)
      )
      (1-5): 5 x FCLayer(
        (linear): WeightNormLinear(in_features=512, out_features=512, bias=True)
      )
    )
    (final_layer): FCLayer(
      (activation_fn): Identity()
      (linear): Linear(in_features=512, out_features=3, bias=True)
    )
  )
)

# How to evaluate **OUR** PyTorch flow model

This is why we instantiated it like this.

```
flow_net = FullyConnectedArch(
    input_keys=[Key("x"), Key("y")],
    output_keys=[Key("u"), Key("v"), Key("p")],
    layer_size=512,
    nr_layers=6,
    skip_connections=False,
    activation_fn=Activation.SILU,
    adaptive_activations=False,
    weight_norm=True,
)
```

In [ ]:
import torch

# Define the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



# Finally, how to visualize our results

In [ ]:
# Define the domain
num_samples = 1000  # Number of points to sample
interior_points = hr_geo.sample_interior(num_samples)

x_values = np.array(interior_points['x'])
y_values = np.array(interior_points['y'])
x_grid, y_grid = np.meshgrid(x_values, y_values)

# Flatten the grid arrays to create inputs for the model
x_flat = x_grid.flatten()
y_flat = y_grid.flatten()

sigma_hoop_value = np.ones(1000000)*46*10**6

# Convert to PyTorch tensors
x_tensor = torch.tensor(x_flat, dtype=torch.float32,device=device).unsqueeze(1)
y_tensor = torch.tensor(y_flat, dtype=torch.float32,device=device).unsqueeze(1)
sigma_hoop_tensor = torch.tensor(sigma_hoop_value, dtype=torch.float32, device=device).unsqueeze(1)


# Create input dictionary for the model
inputs = {"x": x_tensor, "y": y_tensor, "sigma_hoop": sigma_hoop_tensor}

In [ ]:
# Pass inputs through the model to get predictions
outputs = elasticity_net(inputs)
u_flat = outputs['u'].detach().cpu().numpy()
v_flat = outputs['v'].detach().cpu().numpy()
sigma_xx_flat = outputs['sigma_xx'].detach().cpu().numpy()
sigma_yy_flat = outputs['sigma_yy'].detach().cpu().numpy()
sigma_xy_flat = outputs['sigma_xy'].detach().cpu().numpy()

# Reshape the outputs back to grid shape
u_grid = u_flat.reshape(x_grid.shape)
v_grid = v_flat.reshape(x_grid.shape)
sigma_xx_grid = sigma_xx.reshape(x_grid.shape)
sigma_yy_grid = sigma_yy.reshape(x_grid.shape)
sigma_xy_grid = sigma_xy.reshape(x_grid.shape)

In [ ]:
plt.figure(figsize=(15, 10))

# Example of scatter plots
plt.subplot(2, 3, 1)
plt.scatter(x_flat, y_flat, c=u_flat, cmap='jet', s=1)
plt.title('u')
plt.colorbar()

plt.subplot(2, 3, 2)
plt.scatter(x_flat, y_flat, c=v_flat, cmap='jet', s=1)
plt.title('v')
plt.colorbar()

plt.subplot(2, 3, 3)
plt.scatter(x_flat, y_flat, c=sigma_xx_flat, cmap='jet', s=1)
plt.title('sigma_xx')
plt.colorbar()

plt.subplot(2, 3, 4)
plt.scatter(x_flat, y_flat, c=sigma_yy_flat, cmap='jet', s=1)
plt.title('sigma_yy')
plt.colorbar()

plt.subplot(2, 3, 5)
plt.scatter(x_flat, y_flat, c=sigma_xy_flat, cmap='jet', s=1)
plt.title('sigma_xy')
plt.colorbar()

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Plot the color maps with 'jet' colormap
fig, axs = plt.subplots(2, 3, figsize=(18, 12))

# Plot u
im0 = axs[0, 0].imshow(u_grid, extent=[-0.5, 0.5, -0.9, 0.9], origin='lower', aspect='auto', cmap='jet')
axs[0, 0].set_title('u')
axs[0, 0].set_xlabel('x')
axs[0, 0].set_ylabel('y')
fig.colorbar(im0, ax=axs[0, 0])

# Plot v
im1 = axs[0, 1].imshow(v_grid, extent=[-0.5, 0.5, -0.9, 0.9], origin='lower', aspect='auto', cmap='jet')
axs[0, 1].set_title('v')
axs[0, 1].set_xlabel('x')
axs[0, 1].set_ylabel('y')
fig.colorbar(im1, ax=axs[0, 1])

# Plot sigma_xx
im2 = axs[0, 2].imshow(sigma_xx_grid, extent=[-0.5, 0.5, -0.9, 0.9], origin='lower', aspect='auto', cmap='jet')
axs[0, 2].set_title('sigma_xx')
axs[0, 2].set_xlabel('x')
axs[0, 2].set_ylabel('y')
fig.colorbar(im2, ax=axs[0, 2])

# Plot sigma_yy
im3 = axs[1, 0].imshow(sigma_yy_grid, extent=[-0.5, 0.5, -0.9, 0.9], origin='lower', aspect='auto', cmap='jet')
axs[1, 0].set_title('sigma_yy')
axs[1, 0].set_xlabel('x')
axs[1, 0].set_ylabel('y')
fig.colorbar(im3, ax=axs[1, 0])

# Plot sigma_xy
im4 = axs[1, 1].imshow(sigma_xy_grid, extent=[-0.5, 0.5, -0.9, 0.9], origin='lower', aspect='auto', cmap='jet')
axs[1, 1].set_title('sigma_xy')
axs[1, 1].set_xlabel('x')
axs[1, 1].set_ylabel('y')
fig.colorbar(im4, ax=axs[1, 1])

# Empty subplot for completeness
axs[1, 2].axis('off')  # Turn off axis for the last empty subplot

plt.tight_layout()
plt.show()

For the results, go to the inferencer folder inside ouptu and visualize .vtp file in paraview.